In [5]:
import tkinter as tk
import cv2
import numpy as np
from tkinter import filedialog
import json
import os
from tflite_runtime.interpreter import Interpreter, load_delegate
import threading
from queue import Queue
import time

class ObjectDetector:
    def __init__(self, model_path, label_path, threshold=0.3):
        self.threshold = threshold
        self.interpreter = Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        self.input_shape = self.input_details[0]['shape'][1:]
        self.class_labels = self.load_class_labels(label_path)

        self.frame_count = 0
        self.start_time = time.time()

    def load_class_labels(self, label_path):
        with open(label_path, 'r') as json_file:
            class_labels = json.load(json_file)
        return class_labels

    def preprocess_frame(self, frame):
        image_resized = cv2.resize(frame, (self.input_shape[0], self.input_shape[1]))
        image_float32 = image_resized.astype(np.float32)
        image_normalized = image_float32 / 255.0
        image_with_batch = np.expand_dims(image_normalized, 0)
        return image_with_batch

    def nms(self, boxes, scores, threshold):
        if len(boxes) == 0:
            return []
        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]
        areas = (x2 - x1 + 1) * (y2 - y1 + 1)
        order = scores.argsort()[::-1]
        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)
            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])
            w = np.maximum(0.0, xx2 - xx1 + 1)
            h = np.maximum(0.0, yy2 - yy1 + 1)
            overlap = (w * h) / areas[order[1:]]
            inds = np.where(overlap <= threshold)[0]
            order = order[inds + 1]
        return keep
    def perform_inference(self, frame, preprocessed_frame, orig_h, orig_w):
        self.interpreter.set_tensor(self.input_details[0]['index'], preprocessed_frame)
        self.interpreter.invoke()
        output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
        output = np.copy(output_data[0])

        boxes = []
        scores = []

        for i in range(output.shape[1]):
            detection = output[:, i]
            x_center, y_center, width, height = detection[:4]
            x1 = max(0, int((x_center - width / 2) * orig_w))
            y1 = max(0, int((y_center - height / 2) * orig_h))
            x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
            y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))
            score = np.max(detection[4:])
            boxes.append([x1, y1, x2, y2])
            scores.append(score)
           # Extract class index here
            

        boxes = np.array(boxes)
        scores = np.array(scores)
        keep = self.nms(boxes, scores,0.8)

        for idx in keep:
            x1, y1, x2, y2 = boxes[idx]
            score =scores[idx]
            detection = output[:, idx]  # Get the current detection
           
            cls = np.argmax(output[:, idx][4:])
            if score >=self.threshold:  # Retrieve class index for the current detection
                class_name = self.class_labels[str(cls)]  # Use the correct class index
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
                cv2.putText(frame, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    

           

            

    def detect_objects(self, frame):
        preprocessed_frame = self.preprocess_frame(frame)
        orig_h, orig_w = frame.shape[:2]
        self.perform_inference(frame, preprocessed_frame, orig_h, orig_w)

    def run_detection(self, cap):
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            start_frame_time = time.time()
            self.detect_objects(frame)
            end_frame_time = time.time()
            self.frame_count += 1
            fps = 1 / (end_frame_time - start_frame_time)
            print(f"Frame: {self.frame_count}, FPS: {fps:.2f}")

            cv2.imshow('Object Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        total_time = time.time() - self.start_time
        average_fps = self.frame_count / total_time
        print(f"Average FPS: {average_fps:.2f}")
        cap.release()
        cv2.destroyAllWindows()


class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Object Detection")
        self.detector = None

        select_button = tk.Button(root, text="Select Video", command=self.open_video)
        select_button.pack()

    def open_video(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            cap = cv2.VideoCapture(file_path)
            self.detector = ObjectDetector(
                model_path='C:\object-detection-coral\yolov8m_saved_model\yolov8m_float32.tflite',
                label_path='C:/object-detection-coral/label_files/labels_coco.json'
            )
            detection_thread = threading.Thread(target=self.detector.run_detection, args=(cap,))
            detection_thread.start()

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


#{s77777i7a}


Frame: 1, FPS: 6.33
Frame: 2, FPS: 5.50
Frame: 3, FPS: 5.52
Frame: 4, FPS: 6.60
Frame: 5, FPS: 6.54
Frame: 6, FPS: 6.30
Frame: 7, FPS: 6.73
Frame: 8, FPS: 7.15
Frame: 9, FPS: 6.62
Frame: 10, FPS: 7.10
Frame: 11, FPS: 7.24
Frame: 12, FPS: 6.86
Frame: 13, FPS: 7.20
Frame: 14, FPS: 7.10
Frame: 15, FPS: 6.66
Frame: 16, FPS: 6.54
Frame: 17, FPS: 6.98
Frame: 18, FPS: 6.69
Frame: 19, FPS: 7.99
Frame: 20, FPS: 6.40
Frame: 21, FPS: 6.96
Frame: 22, FPS: 7.16
Frame: 23, FPS: 6.96
Frame: 24, FPS: 6.10
Frame: 25, FPS: 6.99
Frame: 26, FPS: 6.17
Frame: 27, FPS: 7.70
Frame: 28, FPS: 7.09
Frame: 29, FPS: 6.83
Frame: 30, FPS: 6.83
Frame: 31, FPS: 6.78
Frame: 32, FPS: 6.58
Frame: 33, FPS: 6.51
Frame: 34, FPS: 6.93
Frame: 35, FPS: 6.66
Frame: 36, FPS: 6.42
Frame: 37, FPS: 7.00
Frame: 38, FPS: 7.16
Frame: 39, FPS: 5.43
Frame: 40, FPS: 6.14
Frame: 41, FPS: 7.83
Frame: 42, FPS: 5.25
Frame: 43, FPS: 7.14
Frame: 44, FPS: 7.19
Frame: 45, FPS: 6.66
Frame: 46, FPS: 6.77
Frame: 47, FPS: 6.81
Frame: 48, FPS: 6.96
F

In [1]:
import tkinter as tk
import cv2
import numpy as np
from tkinter import filedialog
import json
import os
from tflite_runtime.interpreter import Interpreter, load_delegate
import threading
from queue import Queue
import time

class ObjectDetector:
    def __init__(self, model_path, label_path, threshold=0.3):
        self.threshold = threshold
        self.interpreter = Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        self.input_shape = self.input_details[0]['shape'][1:]
        self.class_labels = self.load_class_labels(label_path)

        self.frame_count = 0
        self.start_time = time.time()

    def load_class_labels(self, label_path):
        with open(label_path, 'r') as json_file:
            class_labels = json.load(json_file)
        return class_labels

    def preprocess_frame(self, frame):
        image_resized = cv2.resize(frame, (self.input_shape[0], self.input_shape[1]))
        image_float32 = image_resized.astype(np.float32)
        image_normalized = image_float32 / 255.0
        image_with_batch = np.expand_dims(image_normalized, 0)
        return image_with_batch

    def nms(self, boxes, scores, threshold):
        if len(boxes) == 0:
            return []
        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]
        areas = (x2 - x1 + 1) * (y2 - y1 + 1)
        order = scores.argsort()[::-1]
        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)
            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])
            w = np.maximum(0.0, xx2 - xx1 + 1)
            h = np.maximum(0.0, yy2 - yy1 + 1)
            overlap = (w * h) / areas[order[1:]]
            inds = np.where(overlap <= threshold)[0]
            order = order[inds + 1]
        return keep
    
    def perform_inference(self, frame, preprocessed_frame, orig_h, orig_w):
        self.interpreter.set_tensor(self.input_details[0]['index'], preprocessed_frame)
        self.interpreter.invoke()
        output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
        output = np.copy(output_data[0])

        boxes = []
        scores = []

        for i in range(output.shape[1]):
            detection = output[:, i]
            x_center, y_center, width, height = detection[:4]
            x1 = max(0, int((x_center - width / 2) * orig_w))
            y1 = max(0, int((y_center - height / 2) * orig_h))
            x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
            y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))
            score = np.max(detection[4:])
            if score >= self.threshold:
                boxes.append([x1, y1, x2, y2])
                scores.append(score)

        boxes = np.array(boxes)
        scores = np.array(scores)
        keep = self.nms(boxes, scores, 0.5)

        for idx in keep:
            x1, y1, x2, y2 = boxes[idx]
            score = scores[idx]
            detection = output[:, idx]
            cls = np.argmax(output[:, idx][4:])
            class_name = self.class_labels[str(cls)]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
            cv2.putText(frame, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    def detect_objects(self, frame):
        preprocessed_frame = self.preprocess_frame(frame)
        orig_h, orig_w = frame.shape[:2]
        self.perform_inference(frame, preprocessed_frame, orig_h, orig_w)

    def run_detection(self, cap, output_queue):
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            output_queue.put(frame)

        cap.release()
        output_queue.put(None)

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Object Detection")
        self.detector = None

        select_button = tk.Button(root, text="Select Video", command=self.open_video)
        select_button.pack()

    def open_video(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            cap = cv2.VideoCapture(file_path)
            output_queue = Queue()
            self.detector = ObjectDetector(
                model_path='yolov8m_saved_model/yolov8m_integer_quant.tflite',
                label_path='C:/object-detection-coral/label_files/labels_coco.json'
            )
            detection_thread = threading.Thread(target=self.detector.run_detection, args=(cap, output_queue))
            detection_thread.start()
            self.process_output(output_queue)

    def process_output(self, output_queue):
        while True:
            frame = output_queue.get()
            if frame is None:
                break
            self.detector.detect_objects(frame)
            cv2.imshow('Object Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


In [2]:
import cv2
import numpy as np
from tkinter import filedialog
import json
import os
from tflite_runtime.interpreter import Interpreter, load_delegate
import threading
from queue import Queue
import time

class ObjectDetector:
    def __init__(self, model_path, label_path, threshold=0.3):
        self.threshold = threshold
        self.interpreter = Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        self.input_shape = self.input_details[0]['shape'][1:]
        self.class_labels = self.load_class_labels(label_path)

        self.frame_count = 0
        self.start_time = time.time()

    def load_class_labels(self, label_path):
        with open(label_path, 'r') as json_file:
            class_labels = json.load(json_file)
        return class_labels

    def preprocess_frame(self, frame):
        image_resized = cv2.resize(frame, (self.input_shape[0], self.input_shape[1]))
        image_float32 = image_resized.astype(np.float32)
        image_normalized = image_float32 / 255.0
        image_with_batch = np.expand_dims(image_normalized, 0)
        return image_with_batch

    def nms(self, boxes, scores, threshold):
        if len(boxes) == 0:
            return []
        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]
        areas = (x2 - x1 + 1) * (y2 - y1 + 1)
        order = scores.argsort()[::-1]
        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)
            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])
            w = np.maximum(0.0, xx2 - xx1 + 1)
            h = np.maximum(0.0, yy2 - yy1 + 1)
            overlap = (w * h) / areas[order[1:]]
            inds = np.where(overlap <= threshold)[0]
            order = order[inds + 1]
        return keep
    
    def perform_inference(self, frame, preprocessed_frame, orig_h, orig_w):
        self.interpreter.set_tensor(self.input_details[0]['index'], preprocessed_frame)
        self.interpreter.invoke()
        output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
        output = np.copy(output_data[0])

        boxes = []
        scores = []

        for i in range(output.shape[1]):
            detection = output[:, i]
            x_center, y_center, width, height = detection[:4]
            x1 = max(0, int((x_center - width / 2) * orig_w))
            y1 = max(0, int((y_center - height / 2) * orig_h))
            x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
            y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))
            score = np.max(detection[4:])
            if score >= self.threshold:
                boxes.append([x1, y1, x2, y2])
                scores.append(score)

        boxes = np.array(boxes)
        scores = np.array(scores)
        keep = self.nms(boxes, scores, 0.5)

        for idx in keep:
            x1, y1, x2, y2 = boxes[idx]
            score = scores[idx]
            detection = output[:, idx]
            cls = np.argmax(output[:, idx][4:])
            class_name = self.class_labels[str(cls)]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
            cv2.putText(frame, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        self.frame_count += 1

    def get_fps(self):
        current_time = time.time()
        elapsed_time = current_time - self.start_time
        if elapsed_time > 0:
            fps = self.frame_count / elapsed_time
            return fps
        else:
            return 0

    def detect_objects(self, frame):
        preprocessed_frame = self.preprocess_frame(frame)
        orig_h, orig_w = frame.shape[:2]
        self.perform_inference(frame, preprocessed_frame, orig_h, orig_w)

    def run_detection(self, cap, output_queue):
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            output_queue.put(frame)

        cap.release()
        output_queue.put(None)

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Object Detection")
        self.detector = None

        select_button = tk.Button(root, text="Select Video", command=self.open_video)
        select_button.pack()

    def open_video(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            cap = cv2.VideoCapture(file_path)
            output_queue = Queue()
            self.detector = ObjectDetector(
                model_path='yolov8m_saved_model/yolov8m_integer_quant.tflite',
                label_path='C:/object-detection-coral/label_files/labels_coco.json'
            )
            detection_thread = threading.Thread(target=self.detector.run_detection, args=(cap, output_queue))
            detection_thread.start()
            self.process_output(output_queue)

    def process_output(self, output_queue):
        while True:
            frame = output_queue.get()
            if frame is None:
                break
            self.detector.detect_objects(frame)
            cv2.putText(frame, f"FPS: {self.detector.get_fps():.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Object Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


In [1]:
import tkinter as tk
import cv2
import numpy as np
from tkinter import filedialog
import json
import os
from tflite_runtime.interpreter import Interpreter, load_delegate
import threading
import time

class ObjectDetector:
    def __init__(self, model_path, label_path, threshold=0.3):
        self.threshold = threshold
        self.interpreter = Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        self.input_shape = self.input_details[0]['shape'][1:]
        self.class_labels = self.load_class_labels(label_path)

        self.frame_count = 0
        self.start_time = time.time()

    def load_class_labels(self, label_path):
        with open(label_path, 'r') as json_file:
            class_labels = json.load(json_file)
        return class_labels

    def preprocess_frame(self, frame):
        image_resized = cv2.resize(frame, (self.input_shape[0], self.input_shape[1]))
        image_float32 = image_resized.astype(np.float32)
        image_normalized = image_float32 / 255.0
        image_with_batch = np.expand_dims(image_normalized, 0)
        return image_with_batch

    def nms(self, boxes, scores, threshold):
        if len(boxes) == 0:
            return []
        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]
        areas = (x2 - x1 + 1) * (y2 - y1 + 1)
        order = scores.argsort()[::-1]
        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)
            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])
            w = np.maximum(0.0, xx2 - xx1 + 1)
            h = np.maximum(0.0, yy2 - yy1 + 1)
            overlap = (w * h) / areas[order[1:]]
            inds = np.where(overlap <= threshold)[0]
            order = order[inds + 1]
        return keep

    def perform_inference(self, frame, preprocessed_frame, orig_h, orig_w):
        self.interpreter.set_tensor(self.input_details[0]['index'], preprocessed_frame)
        self.interpreter.invoke()
        output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
        output = np.copy(output_data[0])

        filtered_boxes = []
        filtered_scores = []

        for i in range(output.shape[1]):
            detection = output[:, i]
            x_center, y_center, width, height = detection[:4]
            x1 = max(0, int((x_center - width / 2) * orig_w))
            y1 = max(0, int((y_center - height / 2) * orig_h))
            x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
            y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))
            score = np.max(detection[4:])
            cls = np.argmax(detection[4:])  # Extract class index here
            if score >= self.threshold:
                filtered_boxes.append([x1, y1, x2, y2])
                filtered_scores.append(score)

        filtered_boxes = np.array(filtered_boxes)
        filtered_scores = np.array(filtered_scores)
        keep = self.nms(filtered_boxes, filtered_scores, 0.5)

        for idx in keep:
            x1, y1, x2, y2 = filtered_boxes[idx]
            score = filtered_scores[idx]
            detection = output[:, idx]  # Get the current detection
            cls = np.argmax(detection[4:])  # Retrieve class index for the current detection
            class_name = self.class_labels[str(cls)]  # Use the correct class index
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
            cv2.putText(frame, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    def detect_objects(self, frame):
        preprocessed_frame = self.preprocess_frame(frame)
        orig_h, orig_w = frame.shape[:2]
        self.perform_inference(frame, preprocessed_frame, orig_h, orig_w)

    def run_detection(self, cap):
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            start_frame_time = time.time()
            self.detect_objects(frame)
            end_frame_time = time.time()
            self.frame_count += 1
            fps = 1 / (end_frame_time - start_frame_time)
            print(f"Frame: {self.frame_count}, FPS: {fps:.2f}")

            cv2.imshow('Object Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        total_time = time.time() - self.start_time
        average_fps = self.frame_count / total_time
        print(f"Average FPS: {average_fps:.2f}")
        cap.release()
        cv2.destroyAllWindows()


class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Object Detection")
        self.detector = None

        select_button = tk.Button(root, text="Select Video", command=self.open_video)
        select_button.pack()

    def open_video(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            cap = cv2.VideoCapture(file_path)
            self.detector = ObjectDetector(
                model_path='yolov8m_saved_model/yolov8m_integer_quant.tflite',
                label_path='C:/object-detection-coral/label_files/labels_coco.json'
            )
            detection_thread = threading.Thread(target=self.detector.run_detection, args=(cap,))
            detection_thread.start()

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


Frame: 1, FPS: 8.64
Frame: 2, FPS: 7.64
Frame: 3, FPS: 8.15
Frame: 4, FPS: 7.85
Frame: 5, FPS: 8.46
Frame: 6, FPS: 9.85
Frame: 7, FPS: 8.96
Frame: 8, FPS: 8.77
Frame: 9, FPS: 8.89
Frame: 10, FPS: 7.52
Frame: 11, FPS: 9.30
Frame: 12, FPS: 9.91
Frame: 13, FPS: 10.06
Frame: 14, FPS: 9.81
Frame: 15, FPS: 9.46
Frame: 16, FPS: 9.64
Frame: 17, FPS: 9.52
Frame: 18, FPS: 9.17
Frame: 19, FPS: 9.24
Frame: 20, FPS: 9.09
Frame: 21, FPS: 9.50
Frame: 22, FPS: 9.23
Frame: 23, FPS: 9.65
Frame: 24, FPS: 9.38
Frame: 25, FPS: 9.55
Frame: 26, FPS: 9.28
Average FPS: 8.36
Frame: 1, FPS: 9.27
Frame: 2, FPS: 7.91
Frame: 3, FPS: 7.69
Frame: 4, FPS: 9.27
Frame: 5, FPS: 8.73
Frame: 6, FPS: 8.68
Frame: 7, FPS: 8.76
Frame: 8, FPS: 9.09
Frame: 9, FPS: 10.02
Frame: 10, FPS: 9.64
Frame: 11, FPS: 9.35
Frame: 12, FPS: 9.68
Frame: 13, FPS: 8.89
Frame: 14, FPS: 9.40
Frame: 15, FPS: 9.56
Frame: 16, FPS: 8.99
Frame: 17, FPS: 9.23
Frame: 18, FPS: 9.74
Frame: 19, FPS: 10.43
Frame: 20, FPS: 10.14
Frame: 21, FPS: 10.52
Frame: 2

In [6]:
import tkinter as tk
import cv2
import numpy as np
from tkinter import filedialog
import json
import os
from tflite_runtime.interpreter import Interpreter, load_delegate
import threading
from queue import Queue

# Load the TensorFlow Lite model with Edge TPU delegate
model_path = 'yolov8m_saved_model/yolov8m_integer_quant.tflite'
interpreter = Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape'][1:]
#print("Expected Input Dimensions:", input_details[0]['shape'][1:])

# Threshold Setting
threshold = 0.3

# Load the class labels from the JSON file
with open('C:/object-detection-coral/label_files/labels_coco.json', 'r') as json_file:
    class_labels = json.load(json_file)

# Create a tkinter window
root = tk.Tk()
root.title("Object Detection")

# save_image_with_boxes
output_directory = "output_images"
os.makedirs(output_directory, exist_ok=True)

# Queue for preprocessed frames
preprocessed_queue = Queue(maxsize=5)  # Adjust the size according to your needs

# Lock for synchronization
lock = threading.Lock()

# Function to preprocess the frame
def preprocess_frame(frame, input_shape):
    # Resize frame
    image_resized = cv2.resize(frame, (input_shape[0], input_shape[1]))
    # Convert image to float32
    image_float32 = image_resized.astype(np.float32)
    # Normalize image
    image_normalized = image_float32 / 255.0
    # Add batch dimension
    image_with_batch = np.expand_dims(image_normalized, 0)
    return image_with_batch

# Preprocessing thread function
def preprocessing_thread(cap):
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Preprocess the frame
        preprocessed_frame = preprocess_frame(frame, input_shape)
        # Get original frame dimensions
        orig_h, orig_w = frame.shape[:2]
        # Put the preprocessed frame and its dimensions into the queue
        preprocessed_queue.put((preprocessed_frame, frame, orig_h, orig_w))
    cap.release()
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

def nms(boxes, scores, threshold):
    if len(boxes) == 0:
        return []
    
    # Compute the area of the bounding boxes and sort by score
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        overlap = (w * h) / areas[order[1:]]

        inds = np.where(overlap <= threshold)[0]
        order = order[inds + 1]

    return keep
# Function to perform inference on frames
# Function to perform inference on frames with NMS
def inference_thread():
    frame_count = 0
    start_time = cv2.getTickCount()  # Get initial time
    while True:
        if not preprocessed_queue.empty():
            # Get a preprocessed frame from the queue
            preprocessed_frame, frame, orig_h, orig_w = preprocessed_queue.get()
            frame_count += 1

            # Perform inference
            interpreter.set_tensor(input_details[0]['index'], preprocessed_frame)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            output = output[0]

            # Prepare lists to hold filtered boxes and scores
            filtered_boxes = []
            filtered_scores = []

            # Filter boxes based on threshold
            for i in range(output.shape[1]):
                detection = output[:, i]
                x_center, y_center, width, height = detection[:4]
                x1 = max(0, int((x_center - width / 2) * orig_w))
                y1 = max(0, int((y_center - height / 2) * orig_h))
                x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
                y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))

                score = np.max(detection[4:])
                cls = np.argmax(detection[4:])
                if score >= threshold:
                    filtered_boxes.append([x1, y1, x2, y2])
                    filtered_scores.append(score)

            # Apply NMS
            filtered_boxes = np.array(filtered_boxes)
            filtered_scores = np.array(filtered_scores)
            keep = nms(filtered_boxes, filtered_scores,0.5)

            # Draw bounding boxes after NMS
            for idx in keep:
                x1, y1, x2, y2 = filtered_boxes[idx]
                score = filtered_scores[idx]
                class_name = class_labels[str(cls)]  # Obtain the class name using the class index
                # Use lock to ensure safe access to frame
                with lock:
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)  # Draw bounding box
                    cv2.putText(frame, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (0, 255, 0), 2)

            # Display the processed frame
            with lock:
                cv2.imshow('Object Detection', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    # Calculate FPS
    end_time = cv2.getTickCount()
    total_time = (end_time - start_time) / cv2.getTickFrequency()  # in seconds
    fps = frame_count / total_time
    print(f"Average FPS: {fps}")



# Function to open a file dialog and select a video
def open_video():
    file_path = filedialog.askopenfilename()
    if file_path:
        cap = cv2.VideoCapture(file_path)
        # Start a new thread for preprocessing
        threading.Thread(target=preprocessing_thread, args=(cap,), daemon=True).start()
        # Start the inference thread
        threading.Thread(target=inference_thread, daemon=True).start()

# Button to select a video
select_button = tk.Button(root, text="Select Video", command=open_video)
select_button.pack()

# Run the tkinter event loop
root.mainloop()

#optimized with nms


Average FPS: 9.1789444323761


In [6]:
import tkinter as tk
import cv2
import numpy as np
from tkinter import filedialog
import json
import os
from tflite_runtime.interpreter import Interpreter, load_delegate
import threading
from queue import Queue

# Load the TensorFlow Lite model with Edge TPU delegate
model_path = 'yolov8m_saved_model/yolov8m_integer_quant.tflite'
interpreter = Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape'][1:]
#print("Expected Input Dimensions:", input_details[0]['shape'][1:])

# Threshold Setting
threshold = 0.5

# Load the class labels from the JSON file
with open('C:/object-detection-coral/label_files/labels_coco.json', 'r') as json_file:
    class_labels = json.load(json_file)

# Create a tkinter window
root = tk.Tk()
root.title("Object Detection")

# save_image_with_boxes
output_directory = "output_images"
os.makedirs(output_directory, exist_ok=True)

# Queue for preprocessed frames
preprocessed_queue = Queue(maxsize=5)  # Adjust the size according to your needs

# Lock for synchronization
lock = threading.Lock()

# Function to preprocess the frame
def preprocess_frame(frame, input_shape):
    # Resize frame
    image_resized = cv2.resize(frame, (input_shape[0], input_shape[1]))
    # Convert image to float32
    image_float32 = image_resized.astype(np.float32)
    # Normalize image
    image_normalized = image_float32 / 255.0
    # Add batch dimension
    image_with_batch = np.expand_dims(image_normalized, 0)
    return image_with_batch

# Preprocessing thread function
def preprocessing_thread(cap):
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Preprocess the frame
        preprocessed_frame = preprocess_frame(frame, input_shape)
        # Get original frame dimensions
        orig_h, orig_w = frame.shape[:2]
        # Put the preprocessed frame and its dimensions into the queue
        preprocessed_queue.put((preprocessed_frame, frame, orig_h, orig_w))
    cap.release()

# Function to perform inference on frames
def inference_thread():
    frame_count = 0
    start_time = cv2.getTickCount()  # Get initial time
    while True:
        if not preprocessed_queue.empty():
            # Get a preprocessed frame from the queue
            preprocessed_frame, frame, orig_h, orig_w = preprocessed_queue.get()
            frame_count += 1

            # Perform inference
            interpreter.set_tensor(input_details[0]['index'], preprocessed_frame)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            output = output[0]
            
            

            # Draw bounding boxes
            for i in range(output.shape[1]):
                detection = output[:, i]
                x_center, y_center, width, height = detection[:4]
                x1 = max(0, int((x_center - width / 2) * orig_w))
                y1 = max(0, int((y_center - height / 2) * orig_h))
                x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
                y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))

                score = np.max(detection[4:])
                cls = np.argmax(detection[4:])
                if score >= threshold:
                    class_name = class_labels[str(cls)]  # Obtain the class name using the class index
                    # Use lock to ensure safe access to frame
                    print(f"Detected Class: {class_name}") 
                    with lock:
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)  # Draw bounding box
                        cv2.putText(frame, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                    (0, 255, 0), 2)

            # Display the processed frame
            with lock:
                cv2.imshow('Object Detection', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    # Calculate FPS
    end_time = cv2.getTickCount()
    total_time = (end_time - start_time) / cv2.getTickFrequency()  # in seconds
    fps = frame_count / total_time
    print(f"Average FPS: {fps}")

# Function to open a file dialog and select a video
def open_video():
    file_path = filedialog.askopenfilename()
    if file_path:
        cap = cv2.VideoCapture(file_path)
        # Start a new thread for preprocessing
        threading.Thread(target=preprocessing_thread, args=(cap,), daemon=True).start()
        # Start the inference thread
        threading.Thread(target=inference_thread, daemon=True).start()

# Button to select a video
select_button = tk.Button(root, text="Select Video", command=open_video)
select_button.pack()

# Run the tkinter event loop
root.mainloop()

#optimized

Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected Class: horse
Detected C

In [4]:
import cv2
import numpy as np
from tkinter import filedialog
import json
import os
from tflite_runtime.interpreter import Interpreter, load_delegate
import threading
from queue import Queue
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import tkinter as tk

# Load the TensorFlow Lite model with Edge TPU delegate
model_path = 'C:\object-detection-coral\yolov8m_saved_model\yolov8m_float32.tflite'
interpreter = Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape'][1:]
#print("Expected Input Dimensions:", input_details[0]['shape'][1:])

# Threshold Setting
threshold = 0.3

# Load the class labels from the JSON file
with open('C:/object-detection-coral/label_files/labels_coco.json', 'r') as json_file:
   class_labels = json.load(json_file)

# Create a tkinter window
root = tk.Tk()
root.title("Object Detection")

# save_image_with_boxes
output_directory = "output_images"
os.makedirs(output_directory, exist_ok=True)

# Queue for preprocessed frames
preprocessed_queue = Queue(maxsize=5)  # Adjust the size according to your needs

# Lock for synchronization
lock = threading.Lock()

# Function to preprocess the frame
def preprocess_frame(frame, input_shape):
   # Resize frame
   image_resized = cv2.resize(frame, (input_shape[0], input_shape[1]))
   # Convert image to float32
   image_float32 = image_resized.astype(np.float32)
   # Normalize image
   image_normalized = image_float32 / 255.0
   # Add batch dimension
   image_with_batch = np.expand_dims(image_normalized, 0)
   return image_with_batch

# Preprocessing thread function
def preprocessing_thread(cap):
   while cap.isOpened():
       ret, frame = cap.read()
       if not ret:
           break
       # Preprocess the frame
       preprocessed_frame = preprocess_frame(frame, input_shape)
       # Get original frame dimensions
       orig_h, orig_w = frame.shape[:2]
       # Put the preprocessed frame and its dimensions into the queue
       preprocessed_queue.put((preprocessed_frame, frame, orig_h, orig_w))
   cap.release()


def nms(boxes, scores, threshold):
   if len(boxes) == 0:
       return []
   
   # Compute the area of the bounding boxes and sort by score
   x1 = boxes[:, 0]
   y1 = boxes[:, 1]
   x2 = boxes[:, 2]
   y2 = boxes[:, 3]
   areas = (x2 - x1 + 1) * (y2 - y1 + 1)
   order = scores.argsort()[::-1]

   keep = []
   while order.size > 0:
       i = order[0]
       keep.append(i)
       xx1 = np.maximum(x1[i], x1[order[1:]])
       yy1 = np.maximum(y1[i], y1[order[1:]])
       xx2 = np.minimum(x2[i], x2[order[1:]])
       yy2 = np.minimum(y2[i], y2[order[1:]])

       w = np.maximum(0.0, xx2 - xx1 + 1)
       h = np.maximum(0.0, yy2 - yy1 + 1)
       overlap = (w * h) / areas[order[1:]]

       inds = np.where(overlap <= threshold)[0]
       order = order[inds + 1]

   return keep
# Function to perform inference on frames
# Function to perform inference on frames with NMS
def inference_thread():
   frame_count = 0
   start_time = cv2.getTickCount()  # Get initial time
   while True:
       if not preprocessed_queue.empty():
           # Get a preprocessed frame from the queue
           preprocessed_frame, frame, orig_h, orig_w = preprocessed_queue.get()
           frame_count += 1

           # Perform inference
           interpreter.set_tensor(input_details[0]['index'], preprocessed_frame)
           interpreter.invoke()
           output = interpreter.get_tensor(output_details[0]['index'])
           output = output[0]

           # Prepare lists to hold filtered boxes and scores
           filtered_boxes = []
           filtered_scores = []

           # Filter boxes based on threshold
           for i in range(output.shape[1]):
               detection = output[:, i]
               x_center, y_center, width, height = detection[:4]
               x1 = max(0, int((x_center - width / 2) * orig_w))
               y1 = max(0, int((y_center - height / 2) * orig_h))
               x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
               y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))

               score = np.max(detection[4:])
               cls = np.argmax(detection[4:])
               if score >= threshold:
                   filtered_boxes.append([x1, y1, x2, y2])
                   filtered_scores.append(score)

           # Apply NMS
           filtered_boxes = np.array(filtered_boxes)
           filtered_scores = np.array(filtered_scores)
           keep = nms(filtered_boxes, filtered_scores,0.5)

           # Draw bounding boxes after NMS
           for idx in keep:
               x1, y1, x2, y2 = filtered_boxes[idx]
               score = filtered_scores[idx]
               class_name = class_labels[str(cls)]  # Obtain the class name using the class index
               # Use lock to ensure safe access to frame
               with lock:
                   cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)  # Draw bounding box
                   cv2.putText(frame, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                               (0, 255, 0), 2)

           # Display the processed frame
           with lock:
               cv2.imshow('Object Detection', frame)
               if cv2.waitKey(1) & 0xFF == ord('q'):
                   break

   # Calculate FPS
   end_time = cv2.getTickCount()
   total_time = (end_time - start_time) / cv2.getTickFrequency()  # in seconds
   fps = frame_count / total_time
   print(f"Average FPS: {fps}")



# Function to open a file dialog and select a video
def open_video():
   file_path = filedialog.askopenfilename()
   if file_path:
       cap = cv2.VideoCapture(file_path)
       # Start a new thread for preprocessing
       threading.Thread(target=preprocessing_thread, args=(cap,), daemon=True).start()
       # Start the inference thread
       threading.Thread(target=inference_thread, daemon=True).start()

# Button to select a video
select_button = tk.Button(root, text="Select Video", command=open_video)
select_button.pack()

# Run the tkinter event loop
root.mainloop()

Average FPS: 6.908656947228921


In [1]:
import cv2
import numpy as np
from tkinter import filedialog
import json
import os
from tflite_runtime.interpreter import Interpreter, load_delegate
import threading
from queue import Queue
import time
from sklearn.metrics.pairwise import euclidean_distances
import tkinter as tk

class ObjectDetector:
    def __init__(self, model_path, label_path, threshold=0.3):
        self.threshold = threshold
        self.interpreter = Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        self.input_shape = self.input_details[0]['shape'][1:]
        self.class_labels = self.load_class_labels(label_path)

        self.frame_count = 0
        self.start_time = time.time()

    def load_class_labels(self, label_path):
        with open(label_path, 'r') as json_file:
            class_labels = json.load(json_file)
        return class_labels

    def preprocess_frame(self, frame):
        image_resized = cv2.resize(frame, (self.input_shape[0], self.input_shape[1]))
        image_float32 = image_resized.astype(np.float32)
        image_normalized = image_float32 / 255.0
        image_with_batch = np.expand_dims(image_normalized, 0)
        return image_with_batch

    def nms(self, boxes, scores, threshold):
        if len(boxes) == 0:
            return []
        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]
        areas = (x2 - x1 + 1) * (y2 - y1 + 1)
        order = scores.argsort()[::-1]
        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)
            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])
            w = np.maximum(0.0, xx2 - xx1 + 1)
            h = np.maximum(0.0, yy2 - yy1 + 1)
            overlap = (w * h) / areas[order[1:]]
            inds = np.where(overlap <= threshold)[0]
            order = order[inds + 1]
        return keep
    
    def perform_inference(self, frame, preprocessed_frame, orig_h, orig_w):
        self.interpreter.set_tensor(self.input_details[0]['index'], preprocessed_frame)
        self.interpreter.invoke()
        output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
        output = np.copy(output_data[0])

        boxes = []
        scores = []

        for i in range(output.shape[1]):
            detection = output[:, i]
            x_center, y_center, width, height = detection[:4]
            x1 = max(0, int((x_center - width / 2) * orig_w))
            y1 = max(0, int((y_center - height / 2) * orig_h))
            x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
            y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))
            score = np.max(detection[4:])
            boxes.append([x1, y1, x2, y2])
            scores.append(score)

        boxes = np.array(boxes)
        scores = np.array(scores)
        keep = self.nms(boxes, scores, 0.7)

        for idx in keep:
            x1, y1, x2, y2 = boxes[idx]
            score = scores[idx]
            detection = output[:, idx]
            cls = np.argmax(output[:, idx][4:])
            if score >= self.threshold:
                class_name = self.class_labels[str(cls)]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
                cv2.putText(frame, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        self.frame_count += 1

    def get_fps(self):
        current_time = time.time()
        elapsed_time = current_time - self.start_time
        if elapsed_time > 0:
            fps = self.frame_count / elapsed_time
            return fps
        else:
            return 0

    def detect_objects(self, frame):
        preprocessed_frame = self.preprocess_frame(frame)
        orig_h, orig_w = frame.shape[:2]
        self.perform_inference(frame, preprocessed_frame, orig_h, orig_w)

    def run_detection(self, cap, output_queue):
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            output_queue.put(frame)

        cap.release()
        output_queue.put(None)

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Object Detection")
        self.detector = None

        select_button = tk.Button(root, text="Select Video", command=self.open_video)
        select_button.pack()

    def open_video(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            cap = cv2.VideoCapture(file_path)
            output_queue = Queue()
            self.detector = ObjectDetector(
                model_path='yolov8m_saved_model\yolov8m_integer_quant.tflite',
                label_path='C:/object-detection-coral/label_files/labels_coco.json'
            )
            detection_thread = threading.Thread(target=self.detector.run_detection, args=(cap, output_queue))
            detection_thread.start()
            self.process_output(output_queue)

    def process_output(self, output_queue):
        while True:
            frame = output_queue.get()
            if frame is None:
                break
            self.detector.detect_objects(frame)
            cv2.putText(frame, f"FPS: {self.detector.get_fps():.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Object Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Ala Arboun\AppData\Local\Temp\ipykernel_15192\613201376.py", line 142, in open_video
    self.process_output(output_queue)
  File "C:\Users\Ala Arboun\AppData\Local\Temp\ipykernel_15192\613201376.py", line 149, in process_output
    self.detector.detect_objects(frame)
  File "C:\Users\Ala Arboun\AppData\Local\Temp\ipykernel_15192\613201376.py", line 110, in detect_objects
    self.perform_inference(frame, preprocessed_frame, orig_h, orig_w)
  File "C:\Users\Ala Arboun\AppData\Local\Temp\ipykernel_15192\613201376.py", line 63, in perform_inference
    self.interpreter.set_tensor(self.input_details[0]['index'], preprocessed_frame)
  File "c:\object-detection-coral\venv-inference\Lib\site-packages\tflite_runtime\interpreter.py", line 720, in set_tensor
    self._interpreter.SetTensor(